In [13]:
#import the modules 
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
from scipy import special
import scipy.stats as stats
from numba import jit
from tqdm.notebook import tqdm
import torch


# Functions required


In [2]:
def calcExpK(delta):
    """Function that calculates exponential of hopping matrix"""
    return [[np.cosh(delta),np.sinh(delta)],[np.sinh(delta),np.cosh(delta)]]

def Mphi(phi, expk, nt):
    """
    Function that updates the Fermion matrix
    
    INPUT:
            phi - phi field (array)
            k - hopping parameter (array)
            nt - number of time steps
                        
    OUTPUT:
            M - fermion matrix (update M) 
            
            This uses space index fastest (like Isle)
    """
    n = len(phi)
    nx = int(n/nt)
    if n == M.shape[0]: # this tests if the phi array has the right dimensions
        for t in range(nt-1): # loop over time bloks
            for x in range(nx): # loop over cords and kappa matrix
                M[t*nx+x][t*nx+x] = 1.0 + 0j # diagonal term
                for y in range(nx): # run over the kappa matrix
                    M[(t+1)*nx+x][t*nx+y] = -expk[x][y]*np.exp(1j*phi[t*nx+y]) # off-diagonal
        for x in range(nx): 
            M[(nt-1)*nx+x][(nt-1)*nx+x] = 1.0+0j # diagonal term
            for y in range(nx): # anti-periodic boundary condition
                M[x][(nt-1)*nx+y] = expk[x][y]*np.exp(1j*phi[(nt-1)*nx+y])
        return 0
    else:
        print('# Error! phi and M have inconsistent dimensions!')
        return -1
    
def calcLogDetMM(phi, expk, nt):
    """
    Function that calculates the determinant of Fermion matrices
    
    INPUT:
            phi - phi field (array)
            expk - exponential of hopping parameter (array)
            Nt - number of time steps
                        
    OUTPUT:
            detMM - determinant of the matrices
    """
    
    Mphi(phi, expk, nt) # update M with +1 phi
    detMM = np.log(np.linalg.det(M)) # calc detM with +1 phi
    Mphi(-np.array(phi), expk, nt) # update M with -1 phi
    detMM += np.log(np.linalg.det(M)) # calc detM with -1 phi
    return detMM

def calcTrMM(phi, expk, nt, sign):
    """
    Function that calculates the trace of a Fermion matrix
    
    INPUT:
            phi - phi field (array)
            expk - exponential of hopping term (array)
            nt - number of time steps
            sign - sign of phi (+1 / -1)
                        
    OUTPUT:
            TrMM - trace of the matrix
    """
    TrMM = [] # trace container
    n = len(phi)
    nx = int(n/nt)
    Mphi(phi, expk, nt) # update M
    invM = np.linalg.inv(M)  # only need to invert once!
    for t in range(nt-1): # loop over time blocks
        for x in range(nx): # loop over sites  (space is fastest)
            temp = 0 + 0j
            for y in range(nx):
                temp += invM[t*nx+x][(t+1)*nx+y]*expk[y][x]
            TrMM.append(temp*(-sign*1j*np.exp(1j*phi[t*nx+x])))
    for x in range(nx): # anti-periodic boundry conditions
        temp = 0 + 0j
        for y in range(nx):
            temp += invM[(nt-1)*nx+x][y]*expk[y][x]
        TrMM.append(temp*sign*1j*np.exp(1j*phi[(nt-1)*nx+x]))

    return np.array(TrMM)

def artH(p, phi, expk, nt, U):
    """
    Function that calculates the artificial Hamiltonian of the Hubbard model
    
    INPUT:
            p - conjugate momentum (array)
            phi - phi field (array)            
            expk - exponential of hopping array (array)
            Nt - number of timesteps
            U - onsite coupling (reduced quantity = U * delta)
            
            (optional)
            beta = inverse temperature
                        
    OUTPUT:
            H - artificial Hamiltonian
    """
        
    H = .5*(np.array(p)@np.array(p)+np.array(phi)@np.array(phi)/U) 
    
    H -= np.real(calcLogDetMM(phi, expk, nt))
    

    return H




def gradS(phi,U,Nt):
    """function that calculates gradient"""
    return phi/U - 2*np.real(calcTrMM(phi,expk,Nt,1))

def ribbit(p, phi, expk, nt, U, Nmd,eps, surrogate,force,HMC_phi,check_rev, trajLength = 1.):
    """
    Molecular dynamics integrator (Leap frog algorithm)
    
    INPUT:
            p - conjugate momentum (array)
            phi - phi field (array)
            expk - exponential of hopping parameter (array)
            nt - number of timesteps
            Nmd - number of trajectory pieces
            U - onsite coupling
            eps - integration step
            surrogate - NN trained gradient
            (optional)
            trajLength - length of trajectory
            force - stores the gradient
            HMC_phi - stores the phi values in leapfrog
            check_rev - to check reversibility 
            
    OUTPUT:
            (p, phi) - after integration
            force - list of gradients
            HMC_phi - phi values
    """    
    
    p = np.array([p[i] for i in range(len(p))])
    phi = np.array([phi[i] for i in range(len(phi))])
    

    HMC_phi.append(phi)
    phi = phi + 0.5*eps*p # first half step
    HMC_phi.append(phi)
   
    # steps of integration
    if check_rev:
        L = Nmd-1
    else:
        L = int(np.random.uniform(0, 1)* Nmd)
    for _ in range(L):
        if surrogate is None:         
            p -= eps*gradS(phi,Udelta,nt)
            force.append(gradS(phi,Udelta,nt)) 
        else:
            p -= eps*surrogate.gradient(phi)  
            force.append(surrogate.gradient(phi))                        #<====== replaced with NN
        phi = phi + eps*p
        HMC_phi.append(phi)

    # last half step
    if surrogate is None:
        p -= eps*gradS(phi,Udelta,nt)
        force.append(gradS(phi,Udelta,nt)) 
    else:
        p -= eps*surrogate.gradient(phi)   
        force.append(surrogate.gradient(phi))                   #<====== replaced with NN
    phi = phi + 0.5*eps*p
    HMC_phi.append(phi)
    
    if check_rev:
        return p, phi, force, HMC_phi
    else:
        return p, phi




In [3]:
# def leapfrogIsle(p, phi, expk, nt, U, Nmd,eps, surrogate,force,HMC_p,HMC_phi,check_rev, trajLength = 1.):
#     """
#     Molecular dynamics integrator (Leap frog algorithm)
    
#     INPUT:
#             p - conjugate momentum (array)
#             phi - phi field (array)
#             expk - exponential of hopping parameter (array)
#             nt - number of timesteps
#             Nmd - number of trajectory pieces
#             U - onsite coupling
#             eps - integration step
#             surrogate - NN trained gradient
#             (optional)
#             trajLength - length of trajectory
#             force - stores the gradient
#             HMC_phi - stores the phi values in leapfrog
#             check_rev - to check reversibility 
            
#     OUTPUT:
#             (p, phi) - after integration
#             force - list of gradients
#             HMC_p - p values
#             HMC_phi - phi values
#     """    
    
#     p = np.array([p[i] for i in range(len(p))])
#     phi = np.array([phi[i] for i in range(len(phi))])
    
#     HMC_p.append(p)
#     # first half step
#     if surrogate is None:         
#         p = p - 0.5*eps*gradS(phi,Udelta,nt)
#         force.append(gradS(phi,Udelta,nt)) 
#     else:
#         phi_c = torch.from_numpy(phi).double()
#         f = surrogate(phi_c).detach().numpy()
#         p = p - 0.5*eps*f 
#         force.append(f)
       
    
#     HMC_p.append(p)
    
#     HMC_phi.append(phi)
#     phi = phi + eps*p
#     HMC_phi.append(phi)

#     if check_rev:
#         L = Nmd-1
#     else:
#         L = int(np.random.uniform(0, 1)* Nmd)
#     for _ in range(L):
#         if surrogate is None:         
#             p = p - eps*gradS(phi,Udelta,nt)
#             force.append(gradS(phi,Udelta,nt)) 
#         else:
#             phi_c = torch.from_numpy(phi).double()
#             f = surrogate(phi_c).detach().numpy()
#             p = p - eps*f
#             force.append(f)
            
#         HMC_p.append(p)
#         phi = phi + eps*p
#         HMC_phi.append(phi)

#     # last half step
#     if surrogate is None:
#         p = p - 0.5*eps*gradS(phi,Udelta,nt)
#         force.append(gradS(phi,Udelta,nt)) 
#     else:
#         phi_c = torch.from_numpy(phi).double()
#         f = surrogate(phi_c).detach().numpy()
#         p = p - 0.5*eps*f   
#         force.append(f)                #<====== replaced with NN
    
#     HMC_p.append(p)
    
#     if check_rev:
#         return p, phi, force, HMC_phi,HMC_p
#     else:
#         return p, phi




In [4]:

# def HMCIsle(Nmd,eps,surrogate,check_rev):
#     """ 
#     Function performs the HMC
    
#     INPUT:
#     Nmd : Number of molecular dynamic steps
#     eps : integration step
#     surrogate : None for Normal else NNgHMC
#     check_rev : to check reversibilty 

    
    
#     OUTPUT:
#     ensemble : final phi's
#     prob : acceptance probability
#     start_phi : starting phi for each traj
#     end_phi : ending phi for each traj
#     v_start : starting momentum for each traj
#     v_end : ending momentum for each traj
#     force_fin - list of force 
#     Phi_fin   - list of phi's
    
#     """
#     prob = [] # stores probability
#     ensemble = []  # store the individual configurations here

#     # sample phi from normal distribution with sigma = sqrt(u)
#     phi = np.array([np.random.normal(0,usqrt) for i in range(Nt*Nx)])
# #     phi = np.array([-0.10893624,  0.07288043, -0.15987032, -0.78344818, -0.31146751,
# #         0.56577819,  0.30731219,  0.13713599,  0.38019848,  0.1734272 ,
# #         0.26488761, -0.23517609, -0.80272945, -0.84682043, -0.29287054,
# #         0.91092337, -0.56726987,  1.04876914,  0.12113833,  0.31815587,
# #         0.28167287, -0.01425281,  0.45536667, -0.16282461, -0.37421832,
# #        -0.17292403, -0.23555651,  0.13811461,  1.31369839,  0.65773089,
# #        -0.14854645, -1.58039762])
#     start_phi = []   #stored starting phi for each traj
#     end_phi = []     #stored ending phi for each traj
#     v_start = []     #store starting momentum for each traj
#     v_end = []      #store ending momentum for each traj
#     force_fin = []  #store the force for each traj
#     Phi_fin = []  # store the Phi's for each traj
#     V_fin = []
    
    

#     for traj in tqdm(range(nTrajs)):
#         force = []      
#         HMC_phi = []   
#         HMC_p = []
#         # sample momentum from normal distribution w/ sigma = 1
#         initP = np.array([np.random.normal(0,1) for i in range(Nt*Nx)]) 
# #         initP = np.array([-0.52053269,  0.72790131,  0.29426347,  1.31552286, -0.14189525,
# #        -1.18537663, -1.01583461,  0.74162648, -3.42679191, -1.00185264,
# #        -1.32136822,  0.16669873,  0.89197617,  0.43270766,  1.72128555,
# #         0.10207948, -0.17190911,  0.41073223, -0.97118463,  0.53108782,
# #        -0.11224726,  0.01576091,  1.72832388,  0.33961759,  0.02961404,
# #        -0.97115216,  0.26528083,  1.02289295,  0.83260605,  2.05743302,
# #        -0.82335718, -1.14861087])
#         v_start.append(initP)

#         initPhi = phi
#         start_phi.append(initPhi)

#         initH = artH(initP, initPhi, expk, Nt, Udelta) # initial Hamiltonian
        
#         if check_rev:    
#             finP, finPhi,force,HMC_phi,HMC_p = leapfrogIsle(initP, initPhi, expk, Nt, Udelta, Nmd,eps,surrogate,force,HMC_p,HMC_phi,check_rev)
#             force_fin.append(force)
#             Phi_fin.append(HMC_phi)
#             V_fin.append(HMC_p)
#         else:
#             finP, finPhi = leapfrogIsle(initP, initPhi, expk, Nt, Udelta, Nmd,eps,surrogate,force,HMC_p,HMC_phi,check_rev)

#         finH = artH(finP, finPhi, expk, Nt, Udelta) # final hamiltonian
        
#         end_phi.append(finPhi)
#         v_end.append(finP)
        
# #         print("deltaE=",np.real(finH-initH))
# #         print("energy0=",initH)
# #         print("energy1=",finH)
        

#         # accept/reject step
#         if np.random.uniform(0,1) < np.exp(-np.real(finH-initH)): # accept
#             phi = finPhi
#             ensemble.append(phi)
#             prob.append(1.)
            
#         else: # reject
#             ensemble.append(phi)
#             prob.append(0.)

#     prob = np.array(prob)
# #     print("prob=",prob)
    
#     if check_rev:
#         return ensemble, prob, start_phi, end_phi, v_start, v_end, force_fin, Phi_fin,V_fin
#     else:
#         return ensemble, prob


In [5]:

def HMC(Nmd,eps,surrogate,check_rev):
    """ 
    Function performs the HMC
    
    INPUT:
    Nmd : Number of molecular dynamic steps
    eps : integration step
    surrogate : None for Normal else NNgHMC
    check_rev : to check reversibilty 

    
    
    OUTPUT:
    ensemble : final phi's
    prob : acceptance probability
    start_phi : starting phi for each traj
    end_phi : ending phi for each traj
    v_start : starting momentum for each traj
    v_end : ending momentum for each traj
    force_fin - list of force 
    Phi_fin   - list of phi's
    
    """
    prob = [] # stores probability
    ensemble = []  # store the individual configurations here

    # sample phi from normal distribution with sigma = sqrt(u)
    phi = np.array([np.random.normal(0,usqrt) for i in range(Nt*Nx)])
#     phi = np.array([-0.10893624,  0.07288043, -0.15987032, -0.78344818, -0.31146751,
#         0.56577819,  0.30731219,  0.13713599,  0.38019848,  0.1734272 ,
#         0.26488761, -0.23517609, -0.80272945, -0.84682043, -0.29287054,
#         0.91092337, -0.56726987,  1.04876914,  0.12113833,  0.31815587,
#         0.28167287, -0.01425281,  0.45536667, -0.16282461, -0.37421832,
#        -0.17292403, -0.23555651,  0.13811461,  1.31369839,  0.65773089,
#        -0.14854645, -1.58039762])
    start_phi = []   #stored starting phi for each traj
    end_phi = []     #stored ending phi for each traj
    v_start = []     #store starting momentum for each traj
    v_end = []      #store ending momentum for each traj
    force_fin = []  #store the force for each traj
    Phi_fin = []  # store the Phi's for each traj
    

    for traj in tqdm(range(nTrajs)):
        force = []      
        HMC_phi = []   
        # sample momentum from normal distribution w/ sigma = 1
        initP = np.array([np.random.normal(0,1) for i in range(Nt*Nx)])
#         initP = np.array([-0.52053269,  0.72790131,  0.29426347,  1.31552286, -0.14189525,
#        -1.18537663, -1.01583461,  0.74162648, -3.42679191, -1.00185264,
#        -1.32136822,  0.16669873,  0.89197617,  0.43270766,  1.72128555,
#         0.10207948, -0.17190911,  0.41073223, -0.97118463,  0.53108782,
#        -0.11224726,  0.01576091,  1.72832388,  0.33961759,  0.02961404,
#        -0.97115216,  0.26528083,  1.02289295,  0.83260605,  2.05743302,
#        -0.82335718, -1.14861087])
        v_start.append(initP)

        initPhi = phi
        start_phi.append(initPhi)

        initH = artH(initP, initPhi, expk, Nt, Udelta) # initial Hamiltonian
        
        if check_rev:    
            finP, finPhi,force,HMC_phi = ribbit(initP, initPhi, expk, Nt, Udelta, Nmd,eps,surrogate,force,HMC_phi,check_rev)
            force_fin.append(force)
            Phi_fin.append(HMC_phi)
        else:
            finP, finPhi = ribbit(initP, initPhi, expk, Nt, Udelta, Nmd,eps,surrogate,force,HMC_phi,check_rev)

        finH = artH(finP, finPhi, expk, Nt, Udelta) # final hamiltonian
        
        end_phi.append(finPhi)
        v_end.append(finP)
        
#         print("deltaE=",np.real(finH-initH))
#         print("energy0=",initH)
#         print("energy1=",finH)
        
        
        

        # accept/reject step
        if np.random.uniform(0,1) <= np.exp(-np.real(finH-initH)): # accept
            phi = finPhi
            ensemble.append(phi)
            prob.append(1.)
            
        else: # reject
            ensemble.append(phi)
            prob.append(0.)

    prob = np.array(prob)
#     print("prob=",prob)
    
    if check_rev:
        return ensemble, prob, start_phi, end_phi, v_start, v_end, force_fin, Phi_fin
    else:
        return ensemble, prob


# Constants for the functions

In [6]:
# initialize constants for the functions
U=4. # spin coupling
beta=6. # inverse temperature
Nt=16 # number of time steps
Nx = 2 # number of sites
delta = beta/Nt
Udelta = delta*U
usqrt = np.sqrt(Udelta) #reduced U
M = np.identity(Nt*Nx) + 0j # (Nt*Ni) x (Nt*Ni) identity matrix
expk = calcExpK(delta) # calc exp(kappa)
#Nmd = 10# 3-5 for best acceptance >= 70%


In [7]:
# nTrajs = 10000
# actual_HMC,actualHMC_prob = HMC(Nmd=5,eps=1.0/5.0,surrogate = None,check_rev=False)
# print("actual HMC prob",actualHMC_prob.mean())



# Create Training Data

In [8]:
# # Generate training data from radnom sampling of normal distribution
# num_samples = 10000
# training_gaus = np.zeros((num_samples,Nt*Nx))
# gradient_gaus = np.zeros((num_samples,Nt*Nx))

# data_actual = np.array(actual_HMC)
# gradient_actual = np.zeros((len(actual_HMC),Nt*Nx))

# #stores data from Actual HMC and Gaussian distributions
# xx = np.zeros((len(actual_HMC)+ num_samples,Nt*Nx))
# yy= np.zeros((len(actual_HMC)+ num_samples,Nt*Nx))

# for i in range(num_samples): 
#     training_gaus[i,:] = np.random.normal(0,usqrt,Nt*Nx)
#     gradient_gaus[i,:] = gradS(training_gaus[i,:],U*delta,Nt)
#     gradient_actual[i,:] = gradS(data_actual[i,:],U*delta,Nt)
    


# xx[:num_samples,:] = training_gaus[:num_samples,:]
# xx[num_samples:,:] = data_actual[:,:]

# yy[:num_samples,:] = gradient_gaus[:num_samples,:]
# yy[num_samples:,:] = gradient_actual[:,:]



# plt.hist(xx.flatten(),density=True,bins=30,label='training_gaus + HMC')
# plt.legend()
# # plt.savefig("gaus_HMC.pdf")

# Train the NN to approximate gradient with Pytorch

In [9]:
# import torch
# import torch.nn as nn
# import torch.nn.functional as F
# from torch.utils.data import TensorDataset, DataLoader
# from sklearn.metrics import accuracy_score
# from itertools import chain
# from tqdm.notebook import tqdm

# class NNg(nn.Module):
#     def __init__(self, input_dim, hidden_dim, output_dim):
#         """
#         Args:
#         input_dim: int for input dimension
#         hidden_dim: list of int for multiple hidden layers
#         output_dim: list of int for multiple output layers
#         """

#         # calling constructor of parent class
#         super().__init__()

#         # defining the inputs to the first hidden layer
#         self.hid1 = nn.Linear(input_dim, hidden_dim[0])
#         nn.init.normal_(self.hid1.weight, mean=0.0, std=0.01)
#         nn.init.zeros_(self.hid1.bias)
#         self.act1 = nn.ReLU()

#         # defining the inputs to the output layer
#         self.hid2 = nn.Linear(hidden_dim[0], output_dim)
#         nn.init.xavier_uniform_(self.hid2.weight)

#     def forward(self, X):

#         # input and act for layer 1
#         X = self.hid1(X)
#         X = self.act1(X)

#         # input and act for layer 2
#         X = self.hid2(X)
#         return X

# epochs = 80
# batchsize = 150
# #to access a small section of the training data using the array indexing
# inputs,targets = torch.from_numpy(xx).float(),torch.from_numpy(yy).float()
# train_ds = TensorDataset(inputs, targets)
# # split the data into batches
# train_dl = DataLoader(train_ds, batchsize,shuffle=False)
# test_dl = DataLoader(train_ds, batchsize, shuffle= False)
# model_torch = NNg(Nt*Nx,[2*Nt*Nx],Nt*Nx)
# optimizer = torch.optim.Adam(model_torch.parameters(),weight_decay=1e-5)
# criterion = nn.L1Loss()

# # iterate through all the epoch
# for epoch in tqdm(range(epochs)):
#     # go through all the batches generated by dataloader
#     for i, (inputs, targets) in enumerate(train_dl):
#             # clear the gradients
#             optimizer.zero_grad()
#             # compute the model output
#             yhat = model_torch(inputs)
#             # calculate loss
#             loss = criterion(yhat, targets.type(torch.FloatTensor))
#             # credit assignment
#             loss.backward()
#             # update model weights
#             optimizer.step()
#     #Print the progress
#     if (epoch+1) % 10 == 0:
#         print('Epoch [{}/{}], Loss: {:.4f}'.format(epoch +1, epochs, loss.item()))



# Using the NN inside HMC to sample points

In [10]:
# from surrogate import NeuralGrad_pytorch
# grad_hat = NeuralGrad_pytorch(model_torch,False)

# phi_NNg ,NNg_prob = HMC(Nmd=3,eps=1.0/3.0,surrogate = grad_hat,check_rev=False)
# print("actual HMC prob",actualHMC_prob.mean())
# print("NNg_prob=",NNg_prob.mean())

# phi_NNg ,NNg_prob, NNgstart_phi, NNgend_phi, NNgv_start, NNgv_end,NNgforce, NNg_Phi = HMC(Nmd=3,eps=1./3.,surrogate = grad_hat,check_rev=True)
# print("actualHMC_prob=",actualHMC_prob.mean())
# print("NNg_prob=",NNg_prob.mean())



# Check Reversibility

In [66]:
# Reverse Leap frog

def blp(eps,Nmd,startBackV,startBackPhi,revForce,revPhi,surrogate):
    p = np.array([startBackV[i] for i in range(len(startBackV))])
    phi = np.array([startBackPhi[i] for i in range(len(startBackPhi))]) 
    
    revPhi.append(phi)
    phi = phi - 0.5*eps*p 
    revPhi.append(phi)
   
    for i in range(Nmd-1):
        if surrogate is None:
            p +=  eps*gradS(phi,Udelta,Nt)  
            revForce.append(gradS(phi,Udelta,Nt))
        else:
            p +=  eps*surrogate.gradient(phi) 
            revForce.append(surrogate.gradient(phi))
            
        phi = phi - eps*p
        revPhi.append(phi)
        
    if surrogate is None:
            p +=  eps*gradS(phi,Udelta,Nt)  
            revForce.append(gradS(phi,Udelta,Nt))
    else:
            p +=  eps*surrogate.gradient(phi) 
            revForce.append(surrogate.gradient(phi))
   
    
    phi = phi - 0.5*eps*p
    revPhi.append(phi)

    return p,phi,revForce,revPhi

def reverse(eps,Nmd,endPhi,startV,endV,Force,Phi,surrogate):
    HMCRevForce = []
    HMCRevEndV =[]
    HMCRevPhi = []
    for i in tqdm(range(nTrajs)):
        revForce =[]
        revPhi = []
        revEndV,revEndPhi,revForce,revPhi = blp(eps,Nmd,endV[i],endPhi[i],revForce,revPhi,surrogate)
        HMCRevEndV.append(revEndV)

        revPhi.reverse()
        HMCRevPhi.append(revPhi)

        revForce.reverse()
        HMCRevForce.append(revForce)
    
    
    diff_p = np.abs(np.array(HMCRevEndV) - np.array(startV))
    diff_phi = np.abs(np.array(HMCRevPhi) - np.array(Phi))
    diff_force = np.abs(np.array(HMCRevForce) - np.array(Force))

    print("Difference between forward leapfrog phi and reversed leapfrog phi \n",diff_phi)
    print("\nDifference between forward leapfrog momentum and reversed leapfrog momentum\n",diff_p)
    print("\nDifference between forward leapfrog force and reversed leapfrog force \n",diff_force)


In [69]:
from surrogate import NeuralGrad_pytorch
%reload_ext autoreload
%autoreload 2
nTrajs = 10# number of auxiliary fields in the ensemble
Nmd = 3
eps = 1.0/3.0
model_T = torch.jit.load("NNgModel_2sitesU4B6Nt16.pt")
grad_hat = NeuralGrad_pytorch(model_T)



# HMC 
actual_HMC,actualHMC_prob,HMCstart_phi,HMCend_phi,HMCstart_v,HMCend_v,HMC_force,HMC_phi = HMC(Nmd=3,eps=1.0/3.0,surrogate = None,check_rev=True)
reverse(eps,Nmd,HMCend_phi,HMCstart_v,HMCend_v,HMC_force,HMC_phi,surrogate=None)

#NNgHMC 
NNg_phi ,NNg_prob, NNgstart_phi, NNgend_phi, NNgstart_v, NNgend_v,NNgForce, NNgPhi = HMC(Nmd=3,eps=1./3.,surrogate = grad_hat,check_rev=True)
reverse(eps,Nmd,NNgend_phi,NNgstart_v,NNgend_v,NNgForce,NNgPhi,surrogate=grad_hat)



  0%|          | 0/10 [00:00<?, ?it/s]

  0%|          | 0/10 [00:00<?, ?it/s]

Difference between forward leapfrog phi and reversed leapfrog phi 
 [[[2.22044605e-16 0.00000000e+00 4.44089210e-16 ... 2.22044605e-16
   2.22044605e-16 1.11022302e-16]
  [1.66533454e-16 0.00000000e+00 3.88578059e-16 ... 2.22044605e-16
   0.00000000e+00 1.11022302e-16]
  [1.11022302e-16 0.00000000e+00 1.11022302e-16 ... 2.22044605e-16
   0.00000000e+00 0.00000000e+00]
  [0.00000000e+00 0.00000000e+00 0.00000000e+00 ... 0.00000000e+00
   0.00000000e+00 0.00000000e+00]
  [0.00000000e+00 0.00000000e+00 0.00000000e+00 ... 0.00000000e+00
   0.00000000e+00 0.00000000e+00]]

 [[0.00000000e+00 0.00000000e+00 2.22044605e-16 ... 0.00000000e+00
   0.00000000e+00 0.00000000e+00]
  [0.00000000e+00 0.00000000e+00 2.22044605e-16 ... 0.00000000e+00
   0.00000000e+00 0.00000000e+00]
  [0.00000000e+00 0.00000000e+00 2.22044605e-16 ... 0.00000000e+00
   0.00000000e+00 0.00000000e+00]
  [0.00000000e+00 0.00000000e+00 2.22044605e-16 ... 0.00000000e+00
   0.00000000e+00 0.00000000e+00]
  [0.00000000e+00 0.0

  0%|          | 0/10 [00:00<?, ?it/s]

  0%|          | 0/10 [00:00<?, ?it/s]

Difference between forward leapfrog phi and reversed leapfrog phi 
 [[[0.00000000e+00 0.00000000e+00 0.00000000e+00 ... 3.05311332e-16
   0.00000000e+00 0.00000000e+00]
  [0.00000000e+00 0.00000000e+00 0.00000000e+00 ... 2.22044605e-16
   0.00000000e+00 0.00000000e+00]
  [5.55111512e-17 0.00000000e+00 0.00000000e+00 ... 1.11022302e-16
   0.00000000e+00 0.00000000e+00]
  [5.55111512e-17 0.00000000e+00 0.00000000e+00 ... 0.00000000e+00
   0.00000000e+00 0.00000000e+00]
  [0.00000000e+00 0.00000000e+00 0.00000000e+00 ... 0.00000000e+00
   0.00000000e+00 0.00000000e+00]]

 [[0.00000000e+00 2.22044605e-16 0.00000000e+00 ... 2.77555756e-17
   2.22044605e-16 5.55111512e-17]
  [0.00000000e+00 2.22044605e-16 0.00000000e+00 ... 2.77555756e-17
   0.00000000e+00 5.55111512e-17]
  [0.00000000e+00 0.00000000e+00 0.00000000e+00 ... 0.00000000e+00
   0.00000000e+00 0.00000000e+00]
  [0.00000000e+00 0.00000000e+00 0.00000000e+00 ... 0.00000000e+00
   0.00000000e+00 2.77555756e-17]
  [0.00000000e+00 0.0